# Problem 1

## Treat this problem as a set covering one, all possible trips, which are sets of farms, must be listed first. The possible subsets of farms can be calculated under the creteria that the supplies collected in one trip do not exceed the capacity of truck. Considering the fact that pick up supply from a single farm is definitely high cost, the subsets containing only one farm will not be included. The possible set of all farm subsets is:
## $$S=\{\{1,2\},\{1,4\},\{2,3\},\{2,4\},\{3,4\}\}$$
## $x_i$ is the binary variable that is 1 if $i$th member of $S$ is chosen; 0 otherwise.

In [25]:
using JuMP, Gurobi, NamedArrays

# arrays
farms = [:F1, :F2, :F3, :F4]
sets = [:1, :2, :3, :4, :5]

S = NamedArray([:{1,2}, :{1,4}, :{2,3}, :{2,4}, :{3,4}],(sets))

dist = NamedArray([0  5  12 7  15
                   5  0  4  10 7
                   12 4  0  14 20
                   7  10 14 0  8],([:B, :F1, :F2, :F3],[:B, :F1, :F2, :F3, :F4]),("Farms","Farms"))

A = NamedArray([1 1 0 0 0
                1 0 1 1 0
                0 0 1 0 1
                0 1 0 1 1],(farms,sets),("Farms","Sets"))

m = Model(Gurobi.Optimizer)
set_optimizer_attribute(m, "OutputFlag", 0)

# binary variable for whether we choose a subset
@variable(m, x[sets], Bin)

for i in farms
    @constraint(m, sum(A[i,j]x[j] for j in sets ) >= 1)
end

# minimize the total number of sets
@objective(m, Min, sum(x))

optimize!(m)

println("Which trips should we choose?")
for j in sets
    if value(x[j]) == 1
        println("set ", j)
        println("Route", S[j])
    end
end

Academic license - for non-commercial use only
Academic license - for non-commercial use only
Which trips should we choose?
set 2
Route{1, 4}
set 3
Route{2, 3}


In [28]:
## distance is:
println("Distance is:", dist[:B,:F1] + dist[:F1,:F4] + dist[:B,:F2] + dist[:F2,:F3])

Distance is:38


# Problem 2

In [42]:
using JuMP, Gurobi, NamedArrays

# arrays
bakery = [:Donuts, :Bread, :Cookies, :Scones, :Coffeecake]

clean = NamedArray([0  10 6  15 9
                    4  0  6  17 8
                    10 11 0  20 14
                    7  15 6  0  2
                    5  8  7  7  0],(bakery,bakery),("Baked","Baked"))

time = NamedArray([40,32, 50, 28, 47],(bakery))

N = size(clean,1);

In [45]:
function getSubtour(x,start)
    subtour = [start]
    while true
        j = subtour[end]
        for k in bakery
            if x[k,j] == 1
                push!(subtour,k)
                break
            end
        end
        if subtour[end] == start
            break
        end
    end
    return subtour
end

function getAllSubtours(x)
    nodesRemaining = bakery
    subtours = []
    while length(nodesRemaining) > 0
        subtour = getSubtour(x,nodesRemaining[1])
        push!(subtours, subtour)
        nodesRemaining = setdiff(nodesRemaining,subtour)
    end
    return subtours
end


getAllSubtours (generic function with 1 method)

## MTZ reformulation

In [49]:
m = Model(Gurobi.Optimizer)
set_optimizer_attribute(m, "OutputFlag", 0)

@variable(m, x[bakery,bakery], Bin)

@constraint(m, c1[j in bakery], sum( x[i,j] for i in bakery ) == 1)      # one out-edge
@constraint(m, c2[i in bakery], sum( x[i,j] for j in bakery ) == 1)      # one in-edge
@constraint(m, c3[i in bakery], x[i,i] == 0 )                            # no self-loops

@objective(m, Min, sum( x[i,j]*clean[i,j] for i in bakery, j in bakery ))   # minimize total cost

# MTZ variables and constraints
@variable(m, u[bakery])
@constraint(m, c4[i in bakery, j in bakery[2:end]], u[i] - u[j] + N*x[i,j] <= N-1 )

optimize!(m)
xx = value.(x)
subtours = getAllSubtours(xx) 
println("Tour: ", subtours)
println("Tour length: ", objective_value(m) + sum(time[i] for i in bakery ))

Academic license - for non-commercial use only
Academic license - for non-commercial use only
Tour: Any[[:Donuts, :Bread, :Cookies, :Scones, :Coffeecake, :Donuts]]
Tour length: 234.0


## subtour elimination

In [53]:
m = Model(Gurobi.Optimizer)
set_optimizer_attribute(m, "OutputFlag", 0)

@variable(m, x[bakery,bakery], Bin)

@constraint(m, c1[j in bakery], sum( x[i,j] for i in bakery ) == 1)      # one out-edge
@constraint(m, c2[i in bakery], sum( x[i,j] for j in bakery ) == 1)      # one in-edge
@constraint(m, c3[i in bakery], x[i,i] == 0 )                            # no self-loops

@objective(m, Min, sum( x[i,j]*clean[i,j] for i in bakery, j in bakery ))   # minimize total cost

sols = []

for iters = 1:10
    optimize!(m)
    # total  length of current tour
    println("Tour length: ", objective_value(m) + sum(time[i] for i in bakery ))
    xx = value.(x) # save solution
    push!(sols,xx) # save solution
    subtours = getAllSubtours(xx)  # get all the subtours
    display(subtours) 
    sleep(1)
    # get length of the subtour list
    len = length(subtours)
    if len == 1                    
        # solution is just a single tour!
        println("SOLVED!")
        break
    else
        for subtour in subtours
            L = length(subtour)
            # add constraints that cut off each subtour in the list (add two for each subtour)
            @constraint(m, sum( x[subtour[k+1],subtour[k]] for k = 1:L-1 ) <= L-2)
            @constraint(m, sum( x[subtour[k],subtour[k+1]] for k = 1:L-1 ) <= L-2)
        end
    end
end

2-element Array{Any,1}:
 [:Donuts, :Bread, :Cookies, :Donuts]
 [:Scones, :Coffeecake, :Scones]

Academic license - for non-commercial use only
Academic license - for non-commercial use only
Tour length: 227.0


1-element Array{Any,1}:
 [:Donuts, :Bread, :Cookies, :Scones, :Coffeecake, :Donuts]

Tour length: 234.0
SOLVED!
